In [7]:
import pandas as pd
import pymysql
import os
import time
import logging
import traceback
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()

def connect_db():
    return pymysql.connect(
        host="localhost",
        user="root",
        password=os.getenv("MYSQL_PASSWORD"),
        database="cricstats"
    )

def execute_query(query):
    try:
        connection = connect_db()
        cursor = connection.cursor()
        cursor.execute(query)
        result = cursor.fetchall()
        # Convert tuples to string for storage in CSV
        result_str = str(result)
        connection.commit()
        return result_str
    except Exception as e:
        logging.error(f"Error executing query: {query}")
        logging.error(e)
        return "Error executing query"
    finally:
        if 'cursor' in locals(): cursor.close()
        if 'connection' in locals(): connection.close()

def process_sql_file(file_path):
    try:
        df = pd.read_csv(file_path)
        if 'sql_label' not in df.columns:
            raise ValueError("The file does not contain 'sql_label' column.")

        results = []
        for index, row in df.iterrows():
            sql_query = row['sql_label']
            result = execute_query(sql_query)
            results.append(result)

        df['results'] = results
        output_file_path = os.path.splitext(file_path)[0] + '_results.csv'
        df.to_csv(output_file_path, index=False)
        print(f"Results saved to {output_file_path}")
    except Exception as e:
        logging.error(f"Error processing file: {file_path}")
        logging.error(e)
        logging.error(traceback.format_exc())
        print(f"Error processing file: {e}")

def run_script():
    file_path = 'try4.csv'
    if os.path.exists(file_path):
        process_sql_file(file_path)
    else:
        logging.error(f"File not found: {file_path}")
        print(f"File not found: {file_path}")

if __name__ == "__main__":
    logging.basicConfig(
        filename='sql_execution.log',
        level=logging.ERROR,
        format='%(asctime)s:%(levelname)s:%(message)s'
    )
    run_script()


Results saved to try4_results.csv
